In [1]:
import wget
from zipfile import ZipFile
import glob
import os
from PIL import Image
import numpy as np


In [2]:
url = 'https://object.cscs.ch/v1/AUTH_227176556f3c4bb38df9feea4b91200c/hbp-d002272_BigBrainCytoMapping-v1-v2_pub/Deep-Learning/v1/2019_05_01_v1_masks.zip'

In [3]:
volume_dir='../volume_data/'

In [4]:
wget.download(url,volume_dir)

'../volume_data//2019_05_01_v1_masks (1).zip'

['../volume_data/masks/B20_0943.png',
 '../volume_data/masks/B20_1326.png',
 '../volume_data/masks/B20_0911.png',
 '../volume_data/masks/B20_2271.png',
 '../volume_data/masks/B20_2424.png',
 '../volume_data/masks/B20_1027.png',
 '../volume_data/masks/B20_1740.png',
 '../volume_data/masks/B20_2040.png',
 '../volume_data/masks/B20_1062.png',
 '../volume_data/masks/B20_0708.png',
 '../volume_data/masks/B20_0447.png',
 '../volume_data/masks/B20_1299.png',
 '../volume_data/masks/B20_2252.png',
 '../volume_data/masks/B20_1960.png',
 '../volume_data/masks/B20_1052.png',
 '../volume_data/masks/B20_0337.png',
 '../volume_data/masks/B20_1758.png',
 '../volume_data/masks/B20_1226.png',
 '../volume_data/masks/B20_1934.png',
 '../volume_data/masks/B20_1110.png',
 '../volume_data/masks/B20_0570.png',
 '../volume_data/masks/B20_0557.png',
 '../volume_data/masks/B20_0806.png',
 '../volume_data/masks/B20_2288.png',
 '../volume_data/masks/B20_1146.png',
 '../volume_data/masks/B20_0437.png',
 '../volume_

In [5]:
with ZipFile('../volume_data/2019_05_01_v1_masks.zip', 'r') as zipObj:
   # Extract all the contents of zip file in output directory
   zipObj.extractall(volume_dir)


In [216]:
paths=sorted(glob.glob(os.path.join(volume_dir,'masks/*')))
filenames=[os.path.basename(x) for x in paths]

In [330]:
im= Image.open(paths[0])
width, height=im.size
grid_x,grid_y=np.meshgrid(np.arange(width),np.arange(height))
#get block dimensions
y_length=len(paths)
overall_xmin,overall_ymin,overall_xmax,overall_ymax = [np.inf,np.inf,0,0]
skips=100
for p in range(round(len(paths)/skips)):
    im= Image.open(paths[p*skips]).convert('1')
    xmin,ymin,xmax,ymax=im.getbbox()
    overall_xmin = min(overall_xmin, xmin)
    overall_ymin = min(overall_ymin, ymin)
    overall_xmax = max(overall_xmax, xmax)
    overall_ymax = max(overall_ymax, ymax)
    print(overall_ymin)
    
#add margin of 20 pixels
margin=20
overall_xmin,overall_ymin,overall_xmax,overall_ymax = [overall_xmin,overall_ymin,overall_xmax,overall_ymax] + np.array([-margin,-margin,margin,margin])

2766
2411
2403
2357
2357
2357
2357
2357
2357
2357
2317
2317
2303
2258
2258
2258
2258
2258
2258
2258
2258
2258
2258
2258


In [349]:
print(overall_xmin,overall_ymin,overall_xmax,overall_ymax)
print(width-overall_xmin,height-overall_ymin,width-overall_xmax,height-overall_ymax)

1798 2238 5058 3572
4775 3474 1515 2140


94.48804017631468

In [260]:
#set amount we want to scale down by. 
scaling_down=5
#dimorder is y, z x for bigbrain

dimensions_full_res = np.array([y_length, overall_ymax-overall_ymin, overall_xmax-overall_xmin])
#downscaled dimensions, rounded down
scaled_dimensions = np.floor(dimensions_full_res /scaling_down).astype(int)
#create empty block to fill with data
block=np.zeros((scaled_dimensions.astype(int))).astype(int)

In [261]:
for p in range(scaled_dimensions[0]):
    im= Image.open(paths[p*scaling_down])
    #crop image
    cropped=im.crop((overall_xmin,overall_ymin,overall_xmax,overall_ymax))
    scaled=cropped.resize((scaled_dimensions[2].astype(int),scaled_dimensions[1].astype(int)),Image.NEAREST).convert('1')
    block[p]=np.array(scaled)

#invert     
block=np.flip(block,1)


In [350]:
#origin should be -70.02, -58.677, -70.5666. For some reason -60 works well.
original_origin = np.array([-70.02,-58.6777778, -70.5666667])
start_section=int(paths[0][-8:-4])

crop_shift = np.array([start_section*0.02, (height-overall_ymax)*0.021166667, overall_xmin*0.021166667])
new_origin= original_origin + crop_shift

In [351]:
steps=(0.02*scaling_down, 0.021166667*scaling_down, 0.021166667*scaling_down)
out_vol = pyminc.volumeFromData("V1_block.mnc", block, dimnames=("yspace", "zspace", "xspace"), starts=tuple(new_origin), steps=steps, volumeType="uint")
out_vol.writeFile()    